In [3]:
import numpy as np 
import matplotlib.pyplot as plt 
import csv
import math
cm = 1/2.54
textwidth = 13.98611*cm

%matplotlib qt
normalsize = 12
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Cambria",
    "font.size": normalsize
})

def get_ang(X,Y,Ch,q):
    Fa = -abs(Ch/X) - abs(0.515*(Y-q) / (1+abs(Y-q)) - 0.515)
    Fb = Y/X - Fa*X
    return(np.arctan(2*Fa*X + Fb))


In [42]:
# 1D arrays 
def flow(Ch,q,title,xaxis,yaxis,filename):
    W = 300.0
    H = 300.0
    x = np.arange(0,W+0.1,0.1) 
    y = np.arange(-H,H+0.1,0.1) 
    y[-1] = H

    # Meshgrid 
    X,Y = np.meshgrid(x,y)
    print
    
    # Assign vector directions
    # Fa = -abs(Ch/X) - abs(0.515*(Y-q) / (1+abs(Y-q)) - 0.513)
    # # Fa = -abs(Ch/X) - abs(-1 / (1 + np.exp(Y-q)))
    # Fb = Y/X - Fa*X
    Ftheta = get_ang(X,Y,Ch,q)

    Ex = -np.cos(Ftheta)
    Ey = -np.sin(Ftheta)

    start_x = np.arange(0,H+10,10)

    # stream_points = np.column_stack((start_x, np.sqrt(H*H - start_x*start_x)))
    # stream_points = np.append(stream_points, stream_points*[1,-1], axis=0)
    stream_points = np.column_stack((np.linspace(0,W, 30), np.linspace(0,W,30)))
    # stream_points = np.column_stack((np.linspace(0,W/2, 15), np.full(15,0)))
    stream_points = np.append(stream_points, np.column_stack((np.linspace(0,W/2,10), np.full(10,H))), axis=0)
    # stream_points = np.append(stream_points, np.column_stack((np.linspace(0,W,15), np.full(15,H/2))), axis=0)
    # Depict illustration
    fig = plt.figure()
    plt.streamplot(X,Y,Ex,Ey, start_points=stream_points, density=5, linewidth=None, color='black')
    plt.axhline(0,color='black',linestyle=':')
    plt.gca().set_aspect('equal')

    fig.set_figheight(15*cm)
    # plt.title(title, pad=10)
    plt.xlabel(xaxis, fontsize=normalsize-1)
    plt.ylabel(yaxis, fontsize=normalsize-1)
    plt.show
    plt.savefig(filename, format="pdf", bbox_inches="tight")
flow(2,11,'End Effector Movement Path','Horizontal Distance From Target (mm)','Vertical Distance From Target (mm)',"foot_path.pdf")
# flow(0,0,r'End Effector Movement Path',r'Horizontal Distance From Target (mm)',r'Vertical Distance From Target (mm)',"foot_path.pdf")


/tmp/ipykernel_26584/3089148358.py:17: RuntimeWarning: divide by zero encountered in divide
  Fa = -abs(Ch/X) - abs(0.515*(Y-q) / (1+abs(Y-q)) - 0.515)
/tmp/ipykernel_26584/3089148358.py:18: RuntimeWarning: divide by zero encountered in divide
  Fb = Y/X - Fa*X
/tmp/ipykernel_26584/3089148358.py:18: RuntimeWarning: invalid value encountered in multiply
  Fb = Y/X - Fa*X
/tmp/ipykernel_26584/3089148358.py:19: RuntimeWarning: invalid value encountered in multiply
  return(np.arctan(2*Fa*X + Fb))


In [157]:
def get_points(X,Y,Ch,q, switch_x=0,Xadd=0,Yadd=0):
    threshold = 0.5
    x = 0
    y = 0
    points_x = np.array([x])
    points_y = np.array([y])
    switched = False
    while abs(X) > threshold or abs(Y) > threshold:
        if not switched and switch_x != 0 and x > switch_x:
            plt.scatter(x,y,marker='s',s=80, color='black')
            # plt.text(x,y,'Recomputation Point')
            X += Xadd
            Y += Yadd
            switched = True
        Ftheta = get_ang(X,Y,Ch,q)
        Ex = -np.cos(Ftheta)
        Ey = -np.sin(Ftheta)
        X += Ex
        Y += Ey
        x -= Ex
        y += Ey
        points_x = np.append(points_x, x)
        points_y = np.append(points_y, y)
    return points_x, points_y

def plot_trajectory(X,Y,Ch,q, switch_x=0,Xadd=0,Yadd=0,offset1=0,offset2=0,offset3=0,style='-',with_text=True):
    points_x, points_y = get_points(X,Y,Ch,q, switch_x=0,Xadd=0,Yadd=0)

    style_first = style
    target_text_first = 'Target'
    if switch_x != 0:
        style_first = '--'
        target_text_first = 'Initial Target'
    line = plt.plot(points_x, points_y, style_first, color='black')[0]
    plt.scatter(points_x[0], points_y[0],s=80, color='black')
    if switch_x != 0:
        plt.scatter(points_x[-1], points_y[-1],s=80, facecolor='none', edgecolor='black')
    else:
        plt.scatter(points_x[-1], points_y[-1],s=80, color='black')
    if with_text:
        plt.text(points_x[0]+4, points_y[0]+offset1,'Start',horizontalalignment='left', verticalalignment='center')
        plt.text(points_x[-1]-4, points_y[-1]+offset2,target_text_first,horizontalalignment='right', verticalalignment='center')
    if switch_x != 0:
        points_x, points_y = get_points(X,Y,Ch,q, switch_x,Xadd,Yadd)
        plt.plot(points_x, points_y, color='black')
        plt.scatter(points_x[-1], points_y[-1],s=80, color='black')
        plt.text(points_x[-1]-4, points_y[-1]-offset3,'New Target',horizontalalignment='right', verticalalignment='center')

    # plt.title(f'Foot trajectory with Ch: {Ch} and q: {q}')
    plt.xlabel('Horizontal Position (mm)', fontsize=normalsize-1)
    plt.ylabel('Vertical Position (mm)', fontsize=normalsize-1)
    plt.axhline(0,color='black',linestyle=':')
    plt.axvline(0,color='black',linestyle=':')
    # plt.axis('scaled')
    plt.gcf().set_figwidth(textwidth)
    plt.gcf().set_figheight(6.25*cm)
    # plt.gca().set_aspect('equal')

    return line

plt.figure()
p1 = plot_trajectory(X=150,Y=0,Ch=2,q=0, offset1=10, offset2=10)
p2 = plot_trajectory(X=150,Y=0,Ch=2,q=40,style='-.',with_text=False)
p3 = plot_trajectory(X=150,Y=0,Ch=4,q=0,style='--',with_text=False)
plt.legend([p1,p2,p3],['Ch=2 q=0', 'Ch=2 q=40', 'Ch=4 q=0'])
plt.savefig("traj_1.pdf", format="pdf", bbox_inches="tight")
plt.figure()
p1 = plot_trajectory(X=150,Y=-100,Ch=2,q=0, offset1=14)
p2 = plot_trajectory(X=150,Y=-100,Ch=2,q=40,style='-.',with_text=False)
p3 = plot_trajectory(X=150,Y=-100,Ch=4,q=0,style='--',with_text=False)
plt.legend([p1,p2,p3],['Ch=2 q=0', 'Ch=2 q=40', 'Ch=4 q=0'])
plt.savefig("traj_2.pdf", format="pdf", bbox_inches="tight")
plt.figure()
p1 = plot_trajectory(X=150,Y=100,Ch=2,q=0,offset1=-15)
p2 = plot_trajectory(X=150,Y=100,Ch=2,q=40,style='-.',with_text=False)
p3 = plot_trajectory(X=150,Y=100,Ch=4,q=0,style='--',with_text=False)
plt.legend([p1,p2,p3],['Ch=2 q=0', 'Ch=2 q=40', 'Ch=4 q=0'])
plt.savefig("traj_3.pdf", format="pdf", bbox_inches="tight")

# plt.figure()
# p1 = plot_trajectory(X=150,Y=0,Ch=2,q=0)
# p2 = plot_trajectory(X=150,Y=0,Ch=4,q=0,style=':',with_text=False)
# plt.legend([p1,p2],['Ch=2 q=0', 'Ch=4 q=0'], loc='upper center', bbox_to_anchor=(0.5, 1.16), ncol=2)
# plt.savefig("traj_12.pdf", format="pdf", bbox_inches="tight")
# plt.figure()
# p1 = plot_trajectory(X=150,Y=-100,Ch=2,q=0)
# p2 = plot_trajectory(X=150,Y=-100,Ch=4,q=0,style=':',with_text=False)
# plt.legend([p1,p2],['Ch=2 q=0', 'Ch=4 q=0'], loc='upper center', bbox_to_anchor=(0.5, 1.16), ncol=2)
# plt.savefig("traj_22.pdf", format="pdf", bbox_inches="tight")
# plt.figure()
# p1 = plot_trajectory(X=150,Y=100,Ch=2,q=0)
# p2 = plot_trajectory(X=150,Y=100,Ch=4,q=0,style=':',with_text=False)
# plt.legend([p1,p2],['Ch=2 q=0', 'Ch=4 q=0'], loc='upper center', bbox_to_anchor=(0.5, 1.16), ncol=2)
# plt.savefig("traj_32.pdf", format="pdf", bbox_inches="tight")


plt.figure()
plot_trajectory(X=150,Y=0,Ch=2,q=0,switch_x=100,Xadd=50,Yadd=50, offset1=10, offset2=10)
plt.savefig("traj_re1.pdf", format="pdf", bbox_inches="tight")
plt.figure()
plot_trajectory(X=150,Y=100,Ch=2,q=0,switch_x=100,Xadd=-15,Yadd=-140, offset1=-15)
plt.savefig("traj_re2.pdf", format="pdf", bbox_inches="tight")

    


In [24]:
points1 = np.array([(0, 0), (50, 30), (100, 25)])
points2 = np.array([(100, 25), (80, 70), (50, 30)])
# get x and y vectors
x = points1[:,0]
y = points1[:,1]
x2 = points2[:,0]
y2 = points2[:,1]

# calculate polynomial
z = np.polyfit(x, y, 3)
f = np.poly1d(z)
z2 = np.polyfit(x2, y2, 3)
f2 = np.poly1d(z2)


# calculate new x's and y's
x_wanted = np.linspace(x[0], x[1], 50)
y_wanted = f(x_wanted)
x_real2 = np.linspace(x[1], x[2], 50)
y_real2 = f(x_real2)
x_real1 = np.linspace(x2[0], x2[2], 50)
y_real1 = f2(x_real1)

fig = plt.figure()
plt.plot(x_real2, y_real2,'--', color='black')
plt.plot(x_wanted, y_wanted, color='black')
plt.plot(x_real1, y_real1, color='black')
plt.plot(x[0],y[0],'o', color='black')
plt.plot(x[2],y[2],'o', color='black')
plt.plot(x[1],y[1],'s', markersize=7, color='black')
plt.axhline(0,color='black',linestyle=':')
plt.axvline(0,color='black',linestyle=':')
plt.text(x[1]-4, y[1]-10,'Recomputation point')
plt.text(x[0]+11, y[0]+1,'Start')
plt.text(x[-1]-8, y[-1]-10,'Target')
plt.legend(['Desired Arc', 'Generated Arc'])
# plt.gca().set_aspect('equal')

fig.set_figwidth(textwidth*0.9)
fig.set_figheight(5*cm)
# plt.title(r'Problem With Arc Recomputation', pad=10)
plt.xlabel(r'Horizontal Position (mm)', fontsize=normalsize-1)
plt.ylabel(r'Vertical Position (mm)', fontsize=normalsize-1)

# plt.show()
plt.savefig("old_path.pdf", format="pdf", bbox_inches="tight")


/home/epep/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3505: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/epep/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3505: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)


In [115]:
x = np.linspace(-50,50,1000)
Ex = x
def Ey(q):
    return 0.515*(x-q)/(1+abs(x-q))-0.505
fig = plt.figure()
plt.plot(Ex,Ey(-20),'--', color='black')
plt.plot(Ex,Ey(0), '-',color='black')
plt.plot(Ex,Ey(20), '-.', color='black')
plt.axhline(0,linestyle=':',color='black')
plt.axvline(0,linestyle=':',color='black')
plt.legend([r'\(q=-20\)', r'\(q=-0\)', r'\(q=20\)'])

# plt.gca().set_aspect('equal')
# plt.gca().set_ylim([-1,1])

fig.set_size_inches(textwidth,textwidth*0.45)
# plt.title(r'Sigmoid Like \(S(y)\)', pad=10)
plt.xlabel(r'Vertical Separation (mm)', fontsize=normalsize-1)
# plt.ylabel(r'Vertical Position (mm)', fontsize=normalsize-1)

plt.savefig("sigmoid_like.pdf", format="pdf", bbox_inches="tight")

In [17]:
def euler_from_quaternion(x, y, z, w):
        """
        Convert a quaternion into euler angles (roll, pitch, yaw)
        roll is rotation around x in radians (counterclockwise)
        pitch is rotation around y in radians (counterclockwise)
        yaw is rotation around z in radians (counterclockwise)
        """
        t0 = +2.0 * (w * x + y * z)
        t1 = +1.0 - 2.0 * (x * x + y * y)
        roll_x = math.atan2(t0, t1)
     
        t2 = +2.0 * (w * y - z * x)
        t2 = +1.0 if t2 > +1.0 else t2
        t2 = -1.0 if t2 < -1.0 else t2
        pitch_y = math.asin(t2)
     
        t3 = +2.0 * (w * z + x * y)
        t4 = +1.0 - 2.0 * (y * y + z * z)
        yaw_z = math.atan2(t3, t4)
     
        return roll_x, pitch_y, yaw_z # in radians

start = 50
end = -1
fig,ax = plt.subplots(2)

leg_i = 2
x0 = []
y0 = []
z0 = []

x1 = []
y1 = []
z1 = []

x2 = []
y2 = []
z2 = []

x3 = []
y3 = []
z3 = []

x4 = []
y4 = []
z4 = []

x5 = []
y5 = []
z5 = []
height_true = []
test = 'flat'
with open(f'/home/epep/Documents/HexapodMasters/feet_{test}.csv','r') as csvfile:
    plots = csv.reader(csvfile, delimiter = ',') 
    for row in plots:
        x0.append(-round(float(row[0+0*3])*15,2))
        y0.append(round(float(row[1+0*3])*15,2))
        z0.append(round(float(row[2+0*3])*15,2))

        x1.append(-round(float(row[0+1*3])*15,2))
        y1.append(round(float(row[1+1*3])*15,2))
        z1.append(round(float(row[2+1*3])*15,2))

        x2.append(-round(float(row[0+2*3])*15,2))
        y2.append(round(float(row[1+2*3])*15,2))
        z2.append(round(float(row[2+2*3])*15,2))

        x3.append(-round(float(row[0+3*3])*15,2))
        y3.append(round(float(row[1+3*3])*15,2))
        z3.append(round(float(row[2+3*3])*15,2))

        x4.append(-round(float(row[0+4*3])*15,2))
        y4.append(round(float(row[1+4*3])*15,2))
        z4.append(round(float(row[2+4*3])*15,2))

        x5.append(-round(float(row[0+5*3])*15,2))
        y5.append(round(float(row[1+5*3])*15,2))
        z5.append(round(float(row[2+5*3])*15,2))

        height_true.append(round(float(row[18+leg_i])*15,2))

     
ax[0].plot(x0[start:end], y0[start:end], color='r')
ax[0].plot(x1[start:end], y1[start:end], color='orange')
ax[0].plot(x2[start:end], y2[start:end], color='y')
ax[0].plot(x3[start:end], y3[start:end], color='g')
ax[0].plot(x4[start:end], y4[start:end], color='c')
ax[0].plot(x5[start:end], y5[start:end], color='b')
ax[0].set(ylabel=r'Y Position (cm)')
ax[0].set_aspect('equal')

ax[1].plot(x2[start:end],z2[start:end],color='y', label='Foot Height')
ax[1].plot(x2[start:end],height_true[start:end],'--',color='black', label='Terrain Height')
ax[1].legend(bbox_to_anchor=(0.5, 1),loc='lower center',ncol=2)
ax[1].set(xlabel =r'X Position (cm)')
ax[1].set(ylabel=r'Height (cm)')
ax[1].set_aspect('equal')


x = []
y = []
z = []

roll = []
pitch = []
height_set = []
height_true = []
with open(f'/home/epep/Documents/HexapodMasters/body_{test}.csv','r') as csvfile: 
    plots = csv.reader(csvfile, delimiter = ',') 
    for row in plots:
        x.append(-round(float(row[0])*15,2))
        y.append(-round(float(row[1])*15,2))
        z.append(round(float(row[2])*15,2))
        r,p,_ = euler_from_quaternion(float(row[3]), float(row[4]),float(row[5]),float(row[6]))
        roll.append(np.rad2deg(r))
        pitch.append(np.rad2deg(p))
        height_set.append(round(float(row[7])*15,2))
        height_true.append(round(float(row[8])*15,2))

ax[0].plot(x[start:end], y[start:end], color='m')
fig2,ax2 = plt.subplots(2)
bheight, = ax2[0].plot(x[start:end], z[start:end], color='m', label='Body Height')
dbheight, = ax2[0].plot(x[start:end], height_set[start:end],'--', color='black', label='Desired Body Height')
theight, = ax2[0].plot(x[start:end], height_true[start:end],':', color='black', label='Terrain Height')
ax2[1].plot(x[start:end], roll[start:end], ':', color='black', label='Roll')
ax2[1].plot(x[start:end], pitch[start:end], '--', color='black', label='Pitch')

# ax2[0].set(title=r'Body Height and Rotation Against Forward Distance', ylabel=r'Height (cm)')
ax2[0].set(ylabel=r'Height (cm)')
ax2[1].set(ylabel=r'Rotation (Degrees)')
# ax[0].legend(bbox_to_anchor=(1, 0.95),loc='upper right')
# ax[0].legend()

# first_legend = ax[0].legend(handles=[bheight], loc='upper left')
# second_legend = ax[0].legend(handles=[dbheight],bbox_to_anchor=(0, 0.15), loc='lower left')
# ax[0].add_artist(first_legend)
# ax[0].add_artist(second_legend)
# ax[0].legend(handles=[theight], loc='lower right')

# first_legend = ax2[2].legend(handles=[bheight, theight], loc='lower left')
# ax2[2].add_artist(first_legend)
ax2[0].legend(bbox_to_anchor=(0.5, 1), loc='lower center', ncol=3)

ax2[1].legend(bbox_to_anchor=(0.5, 1), loc='lower center', ncol=2)
# ax2[0].margins(0.01,0.01)
# ax[1].margins(0.01,0.01)


# plt.gca().set_aspect('equal')

fig.set_size_inches(textwidth,textwidth*0.86)
# fig.tight_layout()
fig2.set_size_inches(textwidth,textwidth*0.86)
# fig2.tight_layout()
fig2.subplots_adjust(hspace=0.5)
plt.show()

        
fig.savefig(f"{test}_test_feet.pdf", format="pdf", bbox_inches="tight")
fig2.savefig(f"{test}_test_body.pdf", format="pdf", bbox_inches="tight")
